## Урок 1. Линейное пространство

### Линейное пространство. Основные понятия. Часть 1

__1.__ Исследовать на линейную зависимость:
$$f_{1}(x)=e^{x}, f_{2}(x)=1, f_{3}(x)=x+1, f_{4}(x)=x-e^{x}.$$


__Решение__

Заметим, что $f_{1}(x)=-f_{2}(x)+f_{3}(x)-f_{4}(x)=-1+(x+1)-(x-e^x)=e^x$, то есть вектор $f_{1}(x)$ — линейная комбинация векторов $f_{2}(x)$, $f_{3}(x)$ и $f_{4}(x)$, из чего можно сделать вывод, что $f_{1}(x)=e^{x}, f_{2}(x)=1, f_{3}(x)=x+1, f_{4}(x)=x-e^{x}$ линейно зависимы.

__2.__ Исследовать на линейную зависимость:
$$f_{1}(x)=2, f_{2}(x)=x, f_{3}(x)=x^{2}, f_{4}(x)=(x+1)^{2}.$$


__Решение__

Заметим, что $f_{1}(x)=-4f_{2}(x)-2f_{3}(x)+2f_{4}(x)=-4x-2x^2+2(x+1)^2=-4x-2x^2+2x^2+4x+2=2$, то есть вектор $f_{1}(x)$ — линейная комбинация векторов $f_{2}(x)$, $f_{3}(x)$ и $f_{4}(x)$, из чего можно сделать вывод, что $f_{1}(x)=2, f_{2}(x)=x, f_{3}(x)=x^{2}, f_{4}(x)=(x+1)^{2}$ линейно зависимы.

__3.__ Найти координаты вектора $x = (2, 3, 5)\in \mathbb{R}^{3}$ в базисе $b_{1}=(0, 0, 10)$, $b_{2}=(2, 0, 0)$, $b_{3}=(0, 1, 0)$.

__4.__ Найти координаты вектора $3x^{2}-2x+2\in\mathbb{R}^{3}[x]$:

   а) в базисе $1$, $x$, $x^{2}$;

   б) в базисе $x^{2}$, $x-1$, $1$.

__5.__ Установить, является ли линейным подпространством:

а) совокупность всех векторов трехмерного пространства, у которых по крайней мере одна из первых двух координат равна нулю;

б) все векторы, являющиеся линейными комбинациями данных векторов $\{u_{1}, u_{2}, ..., u_{n}\}$.

### Линейное пространство. Основные понятия. Часть 2

__1.__ Найти скалярное произведение векторов $x, y \in \mathbb{R}$:<br>
а) $x=(0,-3, 6),~y=(-4, 7, 9);$<br>
б) $x=(7, -4, 0, 1),~y=(-3, 1, 11, 2).$

__2.__ Найти нормы векторов $(4, 2, 4)$ и $(12, 3, 4)$ и угол между ними.

__3.__ Будет ли линейное пространство евклидовым, если за скалярное произведение принять:<br>
а) произведение длин векторов;<br>
б) утроенное обычное скалярное произведение векторов?
   
__4.__ Какие из нижеперечисленных векторов образуют ортонормированный базис в линейном пространстве $\mathbb{R}^{3}$:<br>
а) $(1,0,0),(0,0,1);$ <br>
б) $(1/\sqrt{2},-1/\sqrt{2},0),(1/\sqrt{2},1/\sqrt{2},0), (0,0,1);$<br>
в) $(1/2, -1/2, 0), (0, 1/2, 1/2), (0,0,1);$<br>
г) $(1,0,0),(0,1,0),(0,0,1)?$